In [3]:
demofolder = '../../mid/mid-demo/'
spikefile = '11101_1.isk'
moviefile = 'walk1_ieee_128x128x16384.raw'

In [2]:
with open(demofolder+spikefile, 'r') as f:
    nframes = 0
    nspikes = 0
    for line in f:
        nframes += 1
        try:
            nspikes += int(line.rstrip())
        except ValueError:
            print("Can't read this line as an int. Guessing zero.")
print(nframes)
print(nspikes)

Can't read this line as an int.
Can't read this line as an int.
16384
20910


In [4]:
with open(demofolder+moviefile,'rb') as f:
    print(type(f))
    count = 0
    bigcount = 0
    for thing in f:
        print(len(thing))
        count += 1
        bigcount += len(thing)
print(bigcount)
print(count)

<class '_io.BufferedReader'>
123532465
604940
4893040
16025108
82979045
31964868
894
3306
48
8431742
268435456
10


In [6]:
print(thing[1])

57


In [1]:
import matplotlib.pyplot as plt
import numpy as np
plt.ioff()

In [26]:
(128*128*16384) == bigcount

True

In [47]:
with open(demofolder+moviefile,'rb') as f:
    frame = f.read(128*128)
flframe = np.zeros(len(frame))
for pix in range(len(frame)):
    flframe[pix] = float(frame[pix])
plt.imshow(flframe.reshape((128,128)),cmap='gray')

In [ ]:
with open(demofolder+moviefile,'rb') as f:
    while True:
        frame = f.read(128*128)
        if not frame:
            break
        flframe = np.zeros(len(frame))
        for pix in range(len(frame)):
            flframe[pix] = float(frame[pix])
        plt.imshow(flframe.reshape((128,128)),cmap='gray')
        plt.show()

In [12]:
plt.show()

In [2]:
def downsample_frame(frame, factor):
    """Downsample by blocking the image. Only works for integer downsampling rates."""
    if int(factor) != factor:
        raise ValueError("Downsampling rate must be an integer.")
    winit, hinit = frame.shape
    wf=int(winit/factor)
    hf=int(hinit/factor)
    out = np.zeros((wf,hf))
    for x in range(wf):
        for y in range(hf):
            out[x,y] = np.mean(frame[x*factor:(x+1)*factor,y*factor:(y+1)*factor])
    return out

In [3]:
class MovieHandler:
    def __init__(self, moviefile=None, spikefile=None, framesinstim=3, downfactor = 4):
        self.framesinstim = framesinstim
        demofolder = '../../mid/mid-demo/'
        self.downfactor = downfactor
        if moviefile is None:
            moviefile = demofolder +'walk1_ieee_128x128x16384.raw'
        if spikefile is None:
            spikefile = demofolder +'11101_1.isk'
        self.load_movie(moviefile)  
        self.load_spikes(spikefile)
        if len(self.spiketrain) != len(self.movie):
            print('Warning. Spike train length '+str(len(self.spiketrain))+ ' is not equal to movie length '+str(len(self.movie))+'.')        
        
    def get_nstim(self):
        return len(self.spiketrain) - (self.framesinstim-1)
    
    def load_spikes(self, file):
        spiketrain = []
        with open(file, 'r') as f:
            for line in f:
                try:
                    spiketrain.append(int(line.rstrip()))
                except ValueError:
                    print("Can't read this line of spike file as an int. Guessing zero.")
                    spiketrain.append(0)
        self.spiketrain = spiketrain
    
    def load_movie(self, file, frameshape = (128,128)):
        framesize = np.prod(frameshape)
        self.movie = []
        with open(file,'rb') as f:
            while True:
                frame = f.read(framesize)
                if not frame:
                    break
                flframe = np.zeros(len(frame))
                for pix in range(len(frame)):
                    flframe[pix] = float(frame[pix])
                flframe = downsample_frame(flframe.reshape(frameshape),self.downfactor)
                self.movie.append(flframe.flatten())     
        self.frameshape = tuple([int(length/self.downfactor) for length in frameshape])

    def play(self, rate=30):
        try:
            self.movie
        except NameError:
            print("No movie loaded.")
            return
        img = None
        for frame in self.movie:
            fr = frame.reshape(self.frameshape)
            if img is None:
                img = plt.imshow(fr,cmap='gray')
            else:
                img.set_data(fr)
            plt.pause(1./rate)
            plt.draw()
            
    def generator(self):
        lags = self.framesinstim
        try: 
            for t in range(len(self.spiketrain)):
                if t >= lags-1:
                    stim = np.zeros((lags,np.prod(self.frameshape)))
                    for l in range(lags):
                         stim[l,:] = self.movie[t-l].flatten()
                    yield stim.flatten(), self.spiketrain[t]
        #try:
        #    for frame, sp in zip(self.movie, self.spiketrain):
        #            yield frame.flatten(), sp
        except NameError:
            print("Looks like you haven't loaded a movie and spiketrain.")
            raise        
            
    def show_strf(self, strf):
        images = strf.reshape((self.framesinstim,)+self.frameshape)
        plt.figure()
        sqrtsubplots = int(np.ceil((np.sqrt(self.framesinstim))))
        for l in range(self.framesinstim):
            plt.subplot(sqrtsubplots, sqrtsubplots, l+1)
            plt.imshow(images[l], cmap='gray', interpolation='nearest')

In [4]:
mh = MovieHandler()

Can't read this line of spike file as an int. Guessing zero.
Can't read this line of spike file as an int. Guessing zero.


In [9]:
mh.play()

C:\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2435: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


KeyboardInterrupt: 

In [2]:
import mid

In [6]:
from imp import reload
reload(mid)

<module 'mid' from 'C:\\Users\\Eric\\Documents\\Berkeley\\Research\\Neuroscience\\DeWeese_Lab_analysis\\mymid\\MID\\mid.py'>

In [17]:
anmid = mid.MID(handler=mh)

Specialized initialization failed. Falling back on first stimulus.


In [32]:
res = anmid.optimize('GA', rate=1e-5)

Info             Gradient norm
6.29796753817  49.2472012837
6.29602547015  54.5693184343
6.30136756178  54.1883710599
6.15484551964  52.7625721114
6.28433794304  52.1701640169
6.2131708798  56.2730121708
6.44393451744  57.0474015551
6.53122028355  54.0266754943
6.56396851843  55.2025619172
6.51116583113  57.3832552332
6.64313095141  62.243007047
6.52804476776  65.7623152277
6.44441186503  68.9724390292
6.32231645892  83.8516143229
6.44138862632  87.5525569695
6.57776574183  93.9621815914
6.59589516686  87.681865799
6.41250856936  90.4683783321
6.17258411374  90.1039136306
6.14369673911  84.4709158069
6.04600823562  83.8700101082
6.00539060071  88.6164152303
6.05858553451  91.9169007707
5.9021596331  99.2082326843
5.76954247811  101.493212676
5.71886226309  101.255310947
5.61575942302  101.579679169
5.70739448388  117.551340176
5.8117580823  116.837775818
5.82789340196  114.097153893
5.55913212278  122.926313937
5.53578265437  119.24551039
5.47106958194  122.322425998
5.62429443217  137

In [33]:
mh.show_strf(anmid.v)

In [34]:
plt.show()

In [3]:
class TestHandler:
    def __init__(self, datasize=5, ndata=5e5):
        self.nstim = int(ndata)
        self.data = np.random.randn(self.nstim,5)
        self.data = self.data/np.sqrt(np.sum(self.data**2,1))[:,np.newaxis]
        self.truerf = np.zeros((datasize))
        self.truerf[0] = 1
        self.generate_spiketrain()
        
    def get_nstim(self):
        return self.nstim
        
    def generate_spiketrain(self):
        ndata = self.nstim
        self.spiketrain = np.zeros((ndata))
        for point in range(ndata):
            rate = self.truerf.dot(self.data[point])
            rate = rate if rate>0 else 0
            self.spiketrain[point] = np.random.binomial(1,rate)
                           
    def generator(self):
        for stim, spike in zip(self.data, self.spiketrain):
            yield stim, spike

In [7]:
th = TestHandler()
testmid = mid.MID(handler=th)

Specialized initialization failed. Falling back on first stimulus.


In [8]:
testresults = testmid.optimize('GA',rate=1,maxiter=20)

Info             Gradient norm
0.0262158525052  0.075021377821
0.0348552352705  0.0843656474574
0.0456328457917  0.0930779389928
0.058543068011  0.0997596811584
0.0734317680605  0.105249306356
0.089549129927  0.107393027398
0.106195051174  0.106533027729
0.122044300546  0.101529354371
0.13631980471  0.0932313046567
0.148114028495  0.0825901828937
0.157017966332  0.0697413778441
0.163327220867  0.0566235520062
0.167179013147  0.0420918621996
0.169218739285  0.0315228162503
0.170274655535  0.0228816838304
0.170747661462  0.0163697055961
0.17092935309  0.00292138914712
0.170942367446  0.00280132392314
0.170954693249  0.00273419746759
0.171001358095  0.00265943627015
0.171001358095  0.00265943627015
Did not converge to desired precision.


In [9]:
testmid.v

array([ -1.08521085e+00,  -2.67973167e-03,  -4.25508411e-05,
         2.53200956e-03,  -1.58737442e-03])

In [18]:
th.truerf

array([ 1.,  0.,  0.,  0.,  0.])

In [10]:
info, grad = testmid.info_grad(testmid.v,neg=False)

In [11]:
grad

array([ -3.19422677e-03,   4.53008849e-04,  -8.95012687e-05,
         5.58564394e-05,  -5.67877378e-05])

In [12]:
grad.dot(testmid.v)

0.0034654309789836557

In [15]:
v = np.array([.4,.2,.2,.2,.2])
grad = np.zeros(5)
epsilon = .01
info, supposedgrad = testmid.info_grad(v, neg=False)
for direc in range(5):
    delta = np.zeros(5)
    delta[direc] = epsilon
    newinfo, _ =  testmid.info_grad(v+delta, neg=False)
    grad[direc] = (newinfo-info)/epsilon
print(supposedgrad)
print(grad)

[ 0.14781714 -0.06893542 -0.06967218 -0.06996106 -0.06930162]
[ 0.20287896 -0.09543404 -0.10106793 -0.09942686 -0.11212613]


In [17]:
info, pv, pvt = testmid.info_and_dists()
abscissa = np.arange(15)
plt.plot(abscissa, pv, 'b.', abscissa, pvt, 'r.', abscissa, pvt/pv, 'g.')
plt.show()

In [22]:
np.log2(th.get_nstim()/th.spiketrain.sum())

2.4119016054862952

In [25]:
alltheinfo = testmid.info(th.truerf, neg=False)
print(alltheinfo)

0.169973562515


In [27]:
expbias = (th.get_nstim()/th.spiketrain.sum())/(2*np.log(2))
print(expbias)

3.83883347406
